**Redes generativas antagónicas. (GAN)**

*Sí, es la tecnología detrás de los "deepfakes" y todas esas aplicaciones virales de intercambio de caras y envejecimiento. Pero los investigadores tenían intenciones más nobles. Generar conjuntos de datos sintéticos para eliminar información privada.  Detección de anomalías. Conducción autónoma. Arte, música*

*Aprende la distribución real de vectores latentes. No asume distribuciones normales gaussianas como las VAE. El generador mapea el ruido aleatorio (!) a una distribución de probabilidad. El discriminador aprende a identificar imágenes reales a partir de imágenes generadas (falsas). El generador está tratando de engañar al discriminador para que piense que sus imágenes son reales. El discriminador está tratando de atrapar el generador. El generador y el discriminador son adversarios, de ahí el nombre. Una vez que el discriminador ya no puede notar la diferencia, hemos terminado (en teoría)*

**Matemáticas elegantes.**

*Esa es la función de pérdida adversarial. Lo llamamos un "juego min-max". El generador está minimizando su pérdida en la creación de imágenes realistas. El discriminador, al mismo tiempo, está maximizando su capacidad para detectar falsificaciones. Es complicado y delicado. El entrenamiento es muy inestable; Un montón de prueba y error / ajuste de hiperparámetros. Colapso del modo. Degradados que se desvanecen*

In [1]:
import tensorflow as tf


# maintain consistent performance
tf.random.set_seed(1)

# confirm GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


*Como antes, cargaremos el conjunto de datos de Fashion MNIST, lo fusionaremos todo ya que no estamos haciendo entrenamiento / prueba, y normalizaremos los datos como lo hicimos con VAE:*

In [2]:
from tensorflow.keras.datasets import fashion_mnist

# load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
import numpy as np

# merge the training and testing sets
dataset = np.concatenate([x_train, x_test], axis=0)
# normalize the images from [0,255] to [0,1]
dataset = np.expand_dims(dataset, -1).astype("float32") / 255

*Ahora remodelaremos los datos a las dimensiones necesarias para las capas de CNN, los barajaremos y los agruparemos por lotes:*

In [4]:
BATCH_SIZE = 64

# convolution layers work 3 channels
dataset = np.reshape(dataset, (-1, 28, 28, 1))
# create a tensorflow dataset object
dataset = tf.data.Dataset.from_tensor_slices(dataset)
# set the batch size otherwise it reads one image at a time
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

*Ahora configuremos el modelo para nuestro generador. Tenga en cuenta el hiperparámetro NOISE_DIM. A partir de ahí tenemos tres capas Conv2DTranspose para trabajar nuestro camino hacia una imagen final generada.*

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

# the generator's input is a noise vector
# hyper-parameter that also requires fine-tuning
NOISE_DIM = 150

# design a generator model with upsampling layers
# in GANs practices, usually the generator has leaky relu activation while the discriminator has relu
generator = keras.models.Sequential([
  keras.layers.InputLayer(input_shape=(NOISE_DIM,)),
  layers.Dense(7*7*256),
  layers.Reshape(target_shape=(7, 7, 256)),
  layers.Conv2DTranspose(256, 3, activation="LeakyReLU", strides=2, padding="same"),
  layers.Conv2DTranspose(128, 3, activation="LeakyReLU", strides=2, padding="same"),
  layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")
])

generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1894144   
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 256)      590080    
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 128)      295040    
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        1153      
 ranspose)                                                       
                                                        

*A continuación, haremos nuestro modelo discriminador, utilizando dos capas Conv2D para reducir el muestreo antes de entrar en una capa densa de 64 neuronas y una caída para evitar el sobreajuste. Su salida es binaria, ya que su trabajo es clasificar las imágenes como "reales" o "falsas"... hasta que ya no puede notar la diferencia.*

In [6]:
discriminator = keras.models.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28, 1)),
  layers.Conv2D(256, 3, activation="relu", strides=2, padding="same"),
  layers.Conv2D(128, 3, activation="relu", strides=2, padding="same"),
  layers.Flatten(),
  layers.Dense(64, activation="relu"),
  layers.Dropout(0.2),
  layers.Dense(1, activation="sigmoid")
])

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 256)       2560      
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         295040    
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 64)                401472    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 699,137
Trainable params: 699,137
Non-tr

*Configure nuestros optimizadores, función de pérdida y métricas de precisión. Vaya, más hiperparámetros: la clave es obtener las tasas de aprendizaje correctas tanto en el generador como en el discriminador. De lo contrario, no será estable.*

In [7]:
optimizerG = keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.5)
optimizerD = keras.optimizers.Adam(learning_rate=0.00003, beta_1=0.5)

# binary classifier (real or fake)
lossFn = keras.losses.BinaryCrossentropy(from_logits=True)

# accuracy metric
gAccMetric = tf.keras.metrics.BinaryAccuracy()
dAccMetric = tf.keras.metrics.BinaryAccuracy()

*Bien, vamos a unirlos. Inicialmente alimentamos nuestro generador con ruido gaussiano. Sus resultados "falsos" se concatenan a datos "reales" introducidos en el discriminador. El discriminador hace todo lo posible para identificar etiquetas reales vs. falsas. Aquí definimos la función de entrenamiento para el discriminador:*

In [8]:
@tf.function
def trainDStep(data):
  # the batch is (32,28,28,1), so extract 32 value
  batchSize = tf.shape(data)[0]
  # create a noise vector as generator input sampled from Gaussian Random Normal
  # As an exercise try sampling from a uniform distribution and observe the difference
  noise = tf.random.normal(shape=(batchSize, NOISE_DIM))

  # concatenate the real and fake labels
  y_true = tf.concat(
    [
      # the original data is real, labeled with 1
      tf.ones(batchSize, 1),
      # the forged data is fake, labeled with 0
      tf.zeros(batchSize, 1)
    ],
    axis=0
  )

  # record the calculated gradients
  with tf.GradientTape() as tape:
    # generate forged samples
    fake = generator(noise)
    # concatenate real data and forged data
    x = tf.concat([data, fake], axis=0)
    # see if the discriminator detects them
    y_pred = discriminator(x)
    # calculate the loss
    discriminatorLoss = lossFn(y_true, y_pred)

  # apply the backward path and update weights
  grads = tape.gradient(discriminatorLoss, discriminator.trainable_weights)
  optimizerD.apply_gradients(zip(grads, discriminator.trainable_weights))

  # report accuracy
  dAccMetric.update_state(y_true, y_pred)

  # return the loss for visualization
  return {
      "discriminator_loss": discriminatorLoss,
      "discriminator_accuracy": dAccMetric.result()
  }

*Y ahora, la función de entrenamiento para el generador. Recuerde que el generador está tratando de no ser atrapado, por lo que quiere ser clasificado como real. Su función de pérdida se basa en la frecuencia con la que el discriminador clasifica sus muestras falsas como reales. Esto está en tensión con el discriminador, que está tratando de encontrar falsificaciones.*

In [9]:
@tf.function
def trainGStep(data):
  batchSize = tf.shape(data)[0]
  noise = tf.random.normal(shape=(batchSize, NOISE_DIM))
  # when training the generator, we want it to maximize the probability that its
  # output is classified as real, remember the min-max game
  y_true = tf.ones(batchSize, 1)

  with tf.GradientTape() as tape:
    y_pred = discriminator(generator(noise))
    generatorLoss = lossFn(y_true, y_pred)

  grads = tape.gradient(generatorLoss, generator.trainable_weights)
  optimizerG.apply_gradients(zip(grads, generator.trainable_weights))

  gAccMetric.update_state(y_true, y_pred)

  return {
      "generator_loss": generatorLoss,
      "generator_accuracy": gAccMetric.result()
  }

*Configuremos una práctica función para visualizar las imágenes generadas mientras entrenamos:*

In [10]:
from matplotlib import pyplot as plt


def plotImages(model):
    images = model(np.random.normal(size=(81, NOISE_DIM)))

    plt.figure(figsize=(9, 9))

    for i, image in enumerate(images):
        plt.subplot(9,9,i+1)
        plt.imshow(np.squeeze(image, -1), cmap="Greys_r")
        plt.axis('off')

    plt.show();

*Ahora es la hora del espectáculo. Dado que el juego de confrontación es un equilibrio delicado, queremos observar los resultados a medida que avanza el entrenamiento. Puede converger y luego divergir de nuevo; Es algo muy delicado. Aquí simplemente revisamos cada lote, entrenamos al discriminador y entrenamos al generador en cada época. Cada dos épocas echaremos un vistazo a algunas imágenes generadas para ver cómo se ven, ya que el generador aprende distribuciones de probabilidad para representar imágenes realistas.*

In [11]:
for epoch in range(30):

  # accumulate the loss to calculate the average at the end of the epoch
  dLossSum = 0
  gLossSum = 0
  dAccSum = 0
  gAccSum = 0
  cnt = 0

  # loop the dataset one batch at a time
  for batch in dataset:

    # train the discriminator
    # remember you could repeat these 2 lines of code for K times
    dLoss = trainDStep(batch)
    dLossSum += dLoss['discriminator_loss']
    dAccSum += dLoss['discriminator_accuracy']

    # train the generator
    gLoss = trainGStep(batch)
    gLossSum += gLoss['generator_loss']
    gAccSum += gLoss['generator_accuracy']

    # increment the counter
    cnt += 1

  # log the performance
  print("E:{}, Loss G:{:0.4f}, Loss D:{:0.4f}, Acc G:%{:0.2f}, Acc D:%{:0.2f}".format(
      epoch,
      gLossSum/cnt,
      dLossSum/cnt,
      100 * gAccSum/cnt,
      100 * dAccSum/cnt
  ))
    
  if epoch % 2 == 0:
    plotImages(generator)

KeyboardInterrupt: 

In [ ]:
images = generator(np.random.normal(size=(81, NOISE_DIM)))

# plot the generated samples
from matplotlib import pyplot as plt

plt.figure(figsize=(9, 9))

for i, image in enumerate(images):
    plt.subplot(9,9,i+1)
    plt.imshow(np.squeeze(image, -1), cmap="Greys_r")
    plt.axis('off')

plt.show();